In [3]:
"""
Coverage inference

# Input layers
- Up to 5x 2D coordinates
- Up to 64x 2D room vertices
- Distance from 
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Union

# Adapted from https://www.kaggle.com/code/sheikhartin/logic-gates-in-pytorch
# User Logistic Regression because output is either True (comply) or False (not comply)
class LogisticRegression(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Feeds the data to the neural network."""
        output = torch.sigmoid(self.linear1(x))
        output = torch.sigmoid(self.linear2(x))
        output = torch.sigmoid(self.linear3(output))
        return output


In [ ]:
def train(model: Union[nn.Module, nn.Sequential], criterion: Union[nn.MSELoss, nn.BCELoss],
          optimizer: Union[torch.optim.SGD, torch.optim.Adam],
          dataset, epochs: int) -> None:          
          #X: torch.Tensor, y: torch.Tensor, epochs: int) -> None:
    """Trains the neural network and reports."""
    epochs_size = len(str(epochs))  # To beautify when reporting
    
    for epoch in range(epochs+1):
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Print the loss in every 5% of epochs
        if epoch % int(epochs * 0.05) == 0:
            print(f'[EPOCH {epoch: >{epochs_size}}/{epochs}] The loss is {loss.item():.5f}')

In [88]:
"""Load & Preprocess training data
"""
import json
import pandas as pd
import numpy as np

file_path = '../fire-synth/dataset/1711285332_0f8ffdf0-5625-4ed6-bb82-89bbf7282225.json'
df = pd.read_json(file_path)
df = pd.json_normalize(df['data'])
#print(df.head())
len(df.iat[1,1])
len(df.iat[2,1])
# May need to reindex rooms verts and extinguishers
# Until they reach 32 length for verts and 4 length for extinguishers
rooms = df["room"]
extinguishers = df["extinguishers"]
#print(rooms[5000])

def hasNegative(vlist):
    """
    Checks if the vertex list has a negative number
    """
    for v in vlist:
        for vc in v:
            if vc < 0: return True
    return False

def padVerts(vlist, num = 36):
    """
    Pads vertices so they're all the same length.
    TODO: use numpy.pad
    """
    while len(vlist) < num: vlist.append([-1,-1])
    return vlist

def padVertList(df, column, length = 36):
    """
    Pads a particular column containing list of 2D points with [-1,-1]
    to indicate that the points are not used.
    """
    #print(length)
    df[column] = df[column].map(lambda x: padVerts(x, length))
    return df

# STEP 1: TRANSFORM ROOM VERTICES
## Double check if its negative
#rds = rooms.apply(lambda x: hasNegative(x))
#rdf = rds.to_frame()
#rdf.query("@rdf['room'] == True")

# If all are non-negative, we can use negative vertices to determine non-existent points.
df = padVertList(df, "room", 36)

# Double check if they're all properly padded
#rds = df["rooms"].apply(lambda x: len(x))
#print(df.iat[0,1])

# STEP 2: TRANSFORM EXTINGUISHER POINTS
df = padVertList(df, "extinguishers", 12)
#print(df["extinguishers"][0])


In [89]:
# Get as numpy arrays
ex_np = df["extinguishers"].to_numpy()
rm_np = df["room"].to_numpy()

# TODO: Transpose vertices such that: 
# [[x1, y1], [x2, y2] ... [xn, yn]] => [[x1, x2, x3... xn], [y1, y2, y3... ,yn]]
ex_all = list()
for p in ex_np:
    ex_all.append(np.transpose(p))

rm_all = list()
for rm in rm_np:
    rm_all.append(np.transpose(rm))

print(ex_all[0])
print(rm_all[0])

[[ 8.70225e+03 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00
  -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00
  -1.00000e+00 -1.00000e+00]
 [ 6.55200e+03 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00
  -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00
  -1.00000e+00 -1.00000e+00]]
[[    0  9357  9357  6738  6738 11655 11655  3369  3369     0    -1    -1
     -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1
     -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1]
 [    0     0  6552  6552  6892  6892 12592 12592  6892  6892    -1    -1
     -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1
     -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1]]


In [ ]:
# Pack data to be put into training cycle


In [ ]:
epochs = 1000
input_dim = 2
hidden_dim = 3
output_dim = 1
learning_rate = 1e-2

In [ ]:
model_coverage = LogisticRegression(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_coverage.parameters(), lr = learning_rate)

In [ ]:
train(model_coverage, criterion, optimizer, dataset, epochs)